In [1]:
# feature extractoring and preprocessing data
from __future__ import print_function
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras
from keras.models import model_from_json
from keras.models import load_model
import warnings

from keras.preprocessing.image import load_img, save_img, img_to_array
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K

# feature extractoring and preprocessing data
warnings.filterwarnings('ignore')


Using TensorFlow backend.


## Start Style Transfer (Load Model and Weights)

In [2]:
## Load model layers
# Model reconstruction from JSON file
with open('model_num.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_num.h5')

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Instructions for updating:
Colocations handled automatically by placer.


## Gram Matrix

In [3]:
def gram_matrix(x):
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2,0)))
    gram = K.dot(features, K.transpose(features))
    return gram

## Style Loss

In [4]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 1
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

## Content Loss

In [5]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

## Total Variation Loss

In [6]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(
            x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(
            x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

## Load Audios

In [ ]:


def load_content_style_images():
        songname = f'./dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'dataset/img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

## loss functions into scalar

In [7]:
# combine these loss functions into a single scalar
loss = K.variable(0.0)
layer_features = outputs_dict['dense_5']
base_image_features = layer_features[0, :]
combination_features = layer_features[2, :]

#content_weight (experimental value we'll set)
content_weight = 5e-2
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['dense_6',
                  'dense_7', 'dense_8',
                  'dense_9']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :]
    combination_features = layer_features[2, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
import ipdb; ipdb_set_trace()
loss += total_variation_weight * total_variation_loss(combination_image)

# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

ValueError: Dimension must be 1 but is 2 for 'transpose' (op: 'Transpose') with input shapes: [256], [2].